In [32]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import os
import collections
from sklearn.model_selection import ShuffleSplit

def SVMPCA(Components,UserName,Window):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+Window+UserName+'train.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)
#     print("Valence \n ")
#     print("Split the data into training/testing sets \n")
    # Split the data into training/testing sets
#     X_train, X_test, y_train, y_test,z_train,z_test = train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)
#     rs = ShuffleSplit(n_splits=1, test_size=.2, random_state=0)
#     train_index, test_index = rs.split(BandsFile).__next__()
#     X_train, y_train,z_train = BandsFile[train_index], ValenceFile[train_index],ArousalFile[train_index]
#     X_test, y_test,z_test= BandsFile[test_index],  ValenceFile[test_index] ,ArousalFile[test_index]
#     print("Feature Scaling \n")
#     Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
#     print("Applying PCA to select features \n")
    
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    #explained_variance=pca.explained_variance_ratio_
    
#     print("Applying SVM PCA classifier \n")
    # SVM Classifier Valence
    clf = SVC()
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
#     cm = confusion_matrix(y_test, y_predict)
    #print(cm)
#     print("Accuracy score of Valence SVM-PCA")
#     print(accuracy_score(y_test, y_predict)*100)
    
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
#     print("Applying PCA to select features \n")
    
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    #explained_variance=pca.explained_variance_ratio_
    
#     print("Applying SVM classifier \n")
    # SVM Classifier
    clf = SVC()
    clf.fit(X_train, z_train)
    z_predict = clf.predict(X_test)
#     if accuracy_score(y_test, y_predict)*100==87.5:
#         print(y_predict)
#     print("Test set Valence")
#     print(y_test)
#     print("Prediction set Valence")
#     print(y_predict)
#     cm = confusion_matrix(z_test, z_predict_0)
    if accuracy_score(y_test, y_predict)*100==62.5:
        print("y_test")
        print(y_test)
        print("y_predict")
        print(y_predict)
        MainXTrain=X_train
        MainZTrain=z_train
        BandsFile = np.genfromtxt('training/'+Window+UserName+'train.csv',delimiter=',')
        pca = PCA(n_components=Components, svd_solver='full')
        pca.fit(BandsFile)
        BandsFile = pca.transform(BandsFile)
        Accurecies=[]
        for i in range(10):
            clf.fit(MainXTrain,MainZTrain)
            file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
            ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',')
            file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
            ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',')
            BandsFile = np.genfromtxt('training/'+Window+UserName+'train.csv',delimiter=',')
            X_train0, X_test0, y_train0, y_test0 , z_train0,z_test0= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)
#             rs = ShuffleSplit(n_splits=1, test_size=.2, random_state=0)
#             train_index, test_index = rs.split(BandsFile).__next__()
#             X_train, y_train,z_train = BandsFile[train_index], ValenceFile[train_index],ArousalFile[train_index]
#             X_test0, y_test,z_test0= BandsFile[test_index],  ValenceFile[test_index] ,ArousalFile[test_index]  
            z_predict_repeat = clf.predict(X_test0)
            print("z_test0 ")
            print(z_test0 )
            print("z_predict_repeat")
            print(z_predict_repeat)
            Accurecies+=[accuracy_score(z_test0,z_predict_repeat)]
        print("100% Test "+str(np.mean(Accurecies)))
        print(collections.Counter(Accurecies))
    
    #print(cm)
#     print("Accuracy score of Arousal SVM-PCA")
#     print(accuracy_score(z_test, z_predict_0)*100)
#     print("Valence test")
#     for i in range (len(y_test)):
#         print(y_test[i])
# #     print("Valence Train")
# #     for i in range (len(y_train)):
# #         print(y_train[i])
#     print("Valence predict")
#     for i in range (len(y_predict)):
#         print(y_predict[i])
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100]



def Run1000(Components,UserName,Window):
    arousal=[]
    valence=[]
    for i in range (10):
        svmpca=SVMPCA(Components,UserName,Window)
        arousal+=[svmpca[0]]
        valence+=[svmpca[1]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
#     return arousal,valence
        

    
if __name__ == "__main__":
    for i in range(10):
        Run1000(25,"s01","256")

Arousal 51.25
Counter({62.5: 3, 50.0: 3, 37.5: 2, 25.0: 1, 75.0: 1})
Valence 42.5
Counter({37.5: 6, 50.0: 4})
y_test
[1 0 0 1 1 1 0 0]
y_predict
[0 1 0 0 1 1 0 0]
z_test0 
[0. 1. 1. 1. 1. 1. 0. 1.]
z_predict_repeat
[1 1 1 1 1 1 1 1]
z_test0 
[1. 0. 1. 0. 1. 0. 1. 1.]
z_predict_repeat
[1 1 1 1 1 1 1 1]
z_test0 
[1. 1. 1. 1. 1. 1. 0. 0.]
z_predict_repeat
[1 1 1 1 1 1 1 1]
z_test0 
[1. 0. 1. 0. 1. 1. 1. 1.]
z_predict_repeat
[1 1 1 1 1 1 1 1]
z_test0 
[1. 0. 0. 0. 1. 1. 0. 1.]
z_predict_repeat
[1 1 1 1 1 1 1 1]
z_test0 
[1. 0. 0. 1. 0. 0. 0. 1.]
z_predict_repeat
[1 1 1 1 1 1 1 1]
z_test0 
[1. 1. 0. 1. 1. 1. 1. 1.]
z_predict_repeat
[1 1 1 1 1 1 1 1]
z_test0 
[1. 0. 1. 0. 1. 1. 1. 0.]
z_predict_repeat
[1 1 1 1 1 1 1 1]
z_test0 
[1. 1. 1. 1. 1. 0. 1. 1.]
z_predict_repeat
[1 1 1 1 1 1 1 1]
z_test0 
[0. 1. 0. 1. 0. 0. 1. 1.]
z_predict_repeat
[1 1 1 1 1 1 1 1]
100% Test 0.6625
Counter({0.75: 3, 0.625: 2, 0.5: 2, 0.875: 2, 0.375: 1})
y_test
[1 0 0 0 0 1 1 0]
y_predict
[1 0 0 1 0 0 1 1]
z_test0 
[